In [1]:
!pip install xlsxwriter

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
ncd_app8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_app8.xlsx')
ncd_att8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_att8.xlsx')
ncd_box = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_box8.xlsx')
ncd_corlt8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corlt8.xlsx')
ncd_corstd8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corstd8.xlsx')
ncd_corstdp8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corstdp8.xlsx')
ncd_cort8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_cort8.xlsx')
ncd_drv8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_drv8.xlsx')
ncd_fin8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_fin8.xlsx')
ncd_pay8 = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_pay8.xlsx')

In [181]:
ncd_box['사용량'] = ncd_box['상품사이즈']
ncd_box['사용량'] = ncd_box['사용량'].fillna(0)
ncd8 = pd.concat([ncd_app8, ncd_att8, ncd_box, ncd_corlt8, ncd_corstd8, ncd_corstdp8, ncd_cort8, ncd_drv8, ncd_fin8, ncd_pay8])
ncd8 = ncd8.reset_index(drop=True)
ncd8 = ncd8[ncd8['사용량'] != 0]
ncd8 = ncd8[ncd8['요금'] != 0]
ncd8 = ncd8[ncd8['회원그룹명'] != '내부회원']

In [183]:
ncd8['구분1'] = 'Business Application'
ncd8['구분2'] = 'Business Applications'
ncd8['구분3'] = ncd8['인스턴스유형코드명']
ncd8['상품명'] = ncd8['미터링유형코드명']
ncd8['플랫폼'] = '민간'
ncd8['리전명_y'] = '한국'
ncd8['리전번호_y'] = 1

In [185]:
def domain_class(group):
    if ((group == 20240701).any() and (group == 20240731).any()):
        return '기존' 
    elif ((group != 20240701).all() and (group != 20240731).all()):
        return '신규/이탈'
    elif (group != 20240701).all():
        return '신규'
    elif (group != 20240731).all():
        return '이탈'
    else :
        return '기타'


ncd8['회원구분'] = ncd8.groupby(['회원번호', '미터링유형코드명'])['미터링일시'].transform(lambda x: domain_class(x))

In [189]:
ncd8['dayCount'] = ncd8.groupby(['회원번호', '미터링유형코드명'])['회원번호'].transform('size')
ncd8['paid user 여부'] = ncd8['요금'].apply(lambda x : 'trial' if x == 0 else 'paid')
ncd8['약정할인금액'] = np.where(
    (ncd8['미터링유형코드명'].isin(['웍스 코어 Standard Plus'])) & (ncd8['약정여부'] == 'Y'), 
    2000, 
    np.where(
        (ncd8['미터링유형코드명'].isin(['웍스 코어 Lite','웍스 코어 Standard','웍스 드라이브 Lite','웍스 드라이브 Standard','웍스 드라이브 Standard Plus'])) & (ncd8['약정여부'] == 'Y'), 
        1000,
        np.where(
            (ncd8['미터링유형코드명'].isin(['웍스 급여'])) & (ncd8['약정여부'] == 'Y'), 
            1500,
            np.where(
                (ncd8['미터링유형코드명'].isin(['웍스 결재'])) & (ncd8['약정여부'] == 'Y'), 
                1000, 
                np.where(
                    (ncd8['미터링유형코드명'].isin(['웍스 근태','웍스 재무'])) & (ncd8['약정여부'] == 'Y'), 
                    50,
                    np.where(
                        (ncd8['미터링유형코드명'].isin(['웍스 코어 Lite','웍스 코어 Standard','웍스 코어 Standard Plus','웍스 드라이브 Lite','웍스 드라이브 Standard','웍스 드라이브 Standard Plus','웍스 결재','웍스 근태','웍스 재무','웍스 급여'])) & (ncd8['약정여부'] == 'N'), 
                        0, 
                        0
                    )
                )
            )
        )
    )
)
ncd8['미터링일시'] = ncd8['미터링일시'].astype(str)
ncd8['월별일수'] = np.where(
    ncd8['미터링일시'].str[4:6].isin(['01', '03', '05', '07', '08', '10', '12']),
    31,
    np.where(
        ncd8['미터링일시'].str[4:6].isin(['04', '06', '09', '11']),
        30,
        np.where(
            (ncd8['미터링일시'].str[0:4].astype(int) % 4 == 0),
            29, 
            28
        )
    )
)

ncd8['일요금(당일)'] = (ncd8['요금']/ncd8['월별일수'])*ncd8['사용량']
ncd8['약정할인 후 요금'] = ncd8['요금'] - ncd8['약정할인금액']
ncd8['약정할인율'] = ncd8['약정할인금액']/ncd8['요금']
ncd8['결합할인 여부'] = 'Y'
ncd8.loc[ncd8['미터링유형코드명'].isin(['웍스 근태', '웍스 재무', '웍스 급여']), '결합할인 여부'] = 'N'
ncd8['결합할인 여부'] = np.where(
    (ncd8.duplicated(subset=['회원번호','미터링일시','paid user 여부','약정여부','결합할인 여부'], keep=False)) & (ncd8['paid user 여부'] == 'paid') & (ncd8['약정여부'] == 'Y') & (ncd8['결합할인 여부'] == 'Y'),
    'Y',
    'N')
ncd8.loc[ncd8['약정여부'] == 'N', '결합할인 여부'] = 'N'
ncd8['미터링일시'] = ncd8['미터링일시'].astype(str)
ncd8['일매출(당일)'] = np.where(
    (ncd8['결합할인 여부'] == 'Y'),
    ((ncd8['요금']/ncd8['월별일수'])*ncd8['사용량']*(1-ncd8['약정할인율']))*0.9,
    (ncd8['요금']/ncd8['월별일수'])*ncd8['사용량']*(1-ncd8['약정할인율']))

In [190]:
ncd8 = ncd8.sort_values(by=['회원번호', '미터링유형코드명', '미터링일시'])
ncd8['일매출(전일)'] = ncd8.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].shift(1)
ncd8['일매출 증가율'] = ((ncd8['일매출(당일)'] - ncd8['일매출(전일)']) / ncd8['일매출(전일)'])
ncd8['누적 일매출'] = ncd8.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].cumsum()
ncd8['월매출(당월)'] = ncd8.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].transform('sum')
ncd8['월요금(당월)'] = ncd8.groupby(['회원번호', '미터링유형코드명'])['일요금(당일)'].transform('sum')
ncd8 = ncd8.drop(['dayCount'], axis = 1)
ncd8 = ncd8.sort_values(by=['회원번호', '미터링유형코드명', '미터링일시'], ascending = [True, False, True])
ncd8 = ncd8.reset_index(drop=True)
#파트너
#ncd8.loc[ncd8['파트너사회원번호']=='파트너회원', '월매출(당월)'] = ncd8.loc[ncd8['회원그룹명']=='파트너회원', '월매출(당월)'] * 0.7


In [195]:
ncd8.loc[ncd8['미터링유형코드명'].str[:5] == '웍스 드라', '청구유형코드명'] = 'WORKBOX'
ncd8.loc[ncd8['미터링유형코드명'].isin(['웍스 결재','웍스 근태','웍스 재무','웍스 급여']), '청구유형코드명'] = 'WORKPLACE'
ncd8 = ncd8[['회원번호','파트너사회원번호','회원그룹코드','청구유형코드명','미터링유형코드명','미터링일시','사용량','약정여부','월매출(당월)','월요금(당월)']]

In [152]:
ncd8['청구유형코드명'].unique()

array(['NAVER WORKS', 'WORKPLACE', 'WORKBOX'], dtype=object)

In [153]:
ncd8

,회원번호,파트너사회원번호,회원그룹코드,청구유형코드명,미터링유형코드명,미터링일시,사용량,약정여부,월매출(당월),월요금(당월)
0,734,NaN,GEN,NAVER WORKS,웍스 코어 Lite,20240801,35.0,Y,105000.000000,140000.000000
1,734,NaN,GEN,NAVER WORKS,웍스 코어 Lite,20240802,35.0,Y,105000.000000,140000.000000
2,734,NaN,GEN,NAVER WORKS,웍스 코어 Lite,20240803,35.0,Y,105000.000000,140000.000000
3,734,NaN,GEN,NAVER WORKS,웍스 코어 Lite,20240804,35.0,Y,105000.000000,140000.000000
4,734,NaN,GEN,NAVER WORKS,웍스 코어 Lite,20240805,35.0,Y,105000.000000,140000.000000
...,...,...,...,...,...,...,...,...,...,...
1065553,3396626,2552351.0,PATR,NAVER WORKS,웍스 코어 Standard Plus,20240830,1.0,N,838.709677,838.709677
1065554,3396626,2552351.0,PATR,NAVER WORKS,웍스 코어 Standard Plus,20240831,1.0,N,838.709677,838.709677
1065555,3397774,NaN,GEN,NAVER WORKS,웍스 코어 Standard,20240830,3.0,Y,1451.612903,1645.161290
1065556,3397774,NaN,GEN,NAVER WORKS,웍스 코어 Standard,20240831,3.0,Y,1451.612903,1645.161290


In [154]:
# ncd8.to_excel('/Users/user/Documents/업무문서/ncdd8.xlsx',index=False)

In [217]:
ncd8_unique = ncd8.drop_duplicates(subset=['회원번호', '미터링유형코드명'], keep='first').reset_index(drop=True)

In [157]:
ncd8_unique.to_excel('/Users/user/Documents/업무문서/ncdd8.xlsx',index=False)

In [203]:
boa8 = pd.read_excel('/Users/user/Documents/BOA 데이터/boa8.xlsx')
boa8 = boa8.drop(0)
ncd8_unique['회원번호'] = ncd8_unique['회원번호'].astype(int)
boa8['회원번호'] = boa8['회원번호'].astype(int)
boa8 = boa8.groupby(['회원번호','회원명','구분3','파트너명','채널'])[['매출금액(당월)','이용금액(당월)']].sum().reset_index()

In [225]:
ncd8_unique[ncd8_unique['회원번호']==2895255]

,회원번호,구분3,월매출(당월),월요금(당월),파트너사회원번호
23534,2895255,NAVER WORKS,5.501282e+08,6.418162e+08,NaN


In [223]:
ncd8_unique.columns = ['회원번호','파트너사회원번호','회원그룹코드','구분3','미터링유형코드명','미터링일시','사용량','약정여부','월매출(당월)','월요금(당월)']
ncd8_unique1 = ncd8_unique.groupby(['회원번호','구분3'])[['월매출(당월)','월요금(당월)']].sum().reset_index()
ncd8_unique = pd.merge(ncd8_unique1, ncd8_unique[['회원번호','구분3','파트너사회원번호']], how = 'left', on = ['회원번호','구분3'])
ncd8_unique['구분3'].unique()

array(['NAVER WORKS', 'WORKPLACE', 'WORKBOX'], dtype=object)

In [227]:
no_boa8 = ncd8_unique[~ncd8_unique['회원번호'].isin(boa8['회원번호'])]
ncd8_unique = ncd8_unique[ncd8_unique['회원번호'].isin(boa8['회원번호'])]

In [229]:
no_ncd8 = boa8[~boa8['회원번호'].isin(ncd8_unique['회원번호'])]
no_ncd8.to_excel('/Users/user/Documents/업무문서/no_ncd8.xlsx',index=False)

In [231]:
no_boa8.to_excel('/Users/user/Documents/업무문서/no_boa8.xlsx',index=False)

In [233]:
merge8 = pd.merge(ncd8_unique, boa8, how = 'left',on = ['회원번호','구분3'])

In [235]:
merge8 = merge8[['회원번호','회원명','구분3','월매출(당월)','매출금액(당월)','월요금(당월)','이용금액(당월)','파트너사회원번호','파트너명','채널']]
merge8.columns = ['회원번호','회원명','구분3','NCD예측월매출','BOA월매출','NCD예측월요금','BOA월요금','파트너사회원번호','파트너명','채널']
pt = merge8[merge8['채널'] == '파트너 매출']
pt['파트너할인율'] = 0
pt.loc[:, '파트너할인율'] = 1 - (pt['BOA월매출'] / pt['NCD예측월매출'])

pt = pt[['회원번호', '구분3', '파트너사회원번호','파트너할인율']]

merge8 = pd.merge(merge8, pt, how='left', on=['회원번호', '구분3'])

merge8['파트너할인율'] = merge8['파트너할인율'].fillna(0)

merge8['NCD예측월매출'] = merge8['NCD예측월매출'] * (1 - merge8['파트너할인율'])
pt

/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_134/4167287618.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt['파트너할인율'] = 0
/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_134/4167287618.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.65196145e-06 7.16501018e-05 0.00000000e+00 ... 6.06060606e-02
 1.04444444e-01 7.32600733e-04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  pt.loc[:, '파트너할인율'] = 1 - (pt['BOA월매출'] / pt['NCD예측월매출'])


,회원번호,구분3,파트너사회원번호,파트너할인율
2,1084,NAVER WORKS,1702545.0,0.000002
5,1313,NAVER WORKS,2888202.0,0.000072
8,1352,NAVER WORKS,2673247.0,0.000000
9,1404,NAVER WORKS,3816.0,0.000045
11,1431,NAVER WORKS,2547958.0,0.000004
...,...,...,...,...
34098,3382350,NAVER WORKS,2697865.0,0.019853
34102,3382671,NAVER WORKS,2697865.0,0.046154
34103,3382775,NAVER WORKS,2697865.0,0.060606
34104,3382859,NAVER WORKS,2697865.0,0.104444


In [249]:
pt.to_excel('/Users/user/Documents/업무문서/pt8.xlsx',index=False)

In [250]:
pt2 = pt.groupby(['구분3','파트너사회원번호'])['파트너할인율'].mean().reset_index()

In [253]:
pt2.to_excel('/Users/user/Documents/업무문서/pt8_2.xlsx',index=False)

In [243]:
merge8.to_excel('/Users/user/Documents/업무문서/merge8_partnerdiscount.xlsx',index=False)

In [244]:
ncd8 = ncd8.sort_values(by= ['회원번호','미터링일시'])

In [245]:
last8 = ncd8.drop_duplicates(subset=['회원번호', '청구유형코드명'], keep='last').reset_index()

In [246]:
churn8 = last8[(last8['약정여부']=='N')&(last8['미터링일시'] != 20240831)]
churn8 = churn8.drop('index',axis=1)

In [247]:
churn8.columns = ['회원번호','회원그룹코드','구분3','상품명','탈퇴일','사용량','약정여부','월매출(당월)','월요금(당월)']
churn8 = churn8.sort_values(by='탈퇴일')
churn8.to_excel('/Users/user/Documents/업무문서/churn8.xlsx',index=False)

ValueError: Length mismatch: Expected axis has 10 elements, new values have 9 elements